# Baseline model

## It predicts the wind farm dependent average electricity production and aggregates predictions over all wind farms.

In [15]:
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

sys.path.append("..")

In [16]:
data = pd.read_csv('../data/GEFCom2014Data/Wind/raw_data_incl_features.csv', \
                    parse_dates= ['TIMESTAMP'],
                    index_col= 'TIMESTAMP' )
                    
data.interpolate(method = 'linear', inplace= True)
data = pd.get_dummies(data, columns = ['WD100CARD','WD10CARD'], drop_first=True)
RSEED = 42

In [17]:
train = data[:'2013-07-01 00:00:00']
test = data['2013-07-01 01:00:00':]


In [23]:
# zones to loop over
zones = np.sort(train.ZONEID.unique()) 

# baseline predictions of all sites will be merged into one DataFrame to calculate the RMSE with respect to the observations of all zones
# finalpred = pd.DataFrame() 
df_results = pd.DataFrame(index = [f'ZONE{zone}' for zone in zones] + ['TOTAL'], 
                          columns = ['BEST_PARAMS','CV','MODEL','FC','TESTSCORE','TRAINSCORE'])
df_results.loc['TOTAL'].TRAINSCORE = 0
df_results.loc['TOTAL'].TESTSCORE = 0
df_results['MODEL'] = 'Baseline'

# loop over all zones
for zone in zones:

    # get train and test data of individual zones
    ytrain = train[train.ZONEID == zone].TARGETVAR
    ytest =  test[test.ZONEID == zone].TARGETVAR

    # baseline predicton for individual zone
    pred_train = np.ones(len(ytrain)) * np.mean(ytrain)
    pred_test = np.ones(len(ytest)) * np.mean(ytrain)

    df_results.loc[f'ZONE{zone}'].TRAINSCORE = mean_squared_error(ytrain, pred_train, squared=False)
    df_results.loc[f'ZONE{zone}'].TESTSCORE = mean_squared_error(ytest, pred_test, squared=False)

    df_results.loc['TOTAL'].TRAINSCORE += np.power(df_results.loc[f'ZONE{zone}'].TRAINSCORE,2) * len(ytrain)/len(train)
    df_results.loc['TOTAL'].TESTSCORE += np.power(df_results.loc[f'ZONE{zone}'].TESTSCORE,2) * len(ytest)/len(test)

df_results.loc['TOTAL'].TRAINSCORE = np.power(df_results.loc['TOTAL'].TRAINSCORE,.5)
df_results.loc['TOTAL'].TESTSCORE = np.power(df_results.loc['TOTAL'].TESTSCORE,.5)

df_results.index.set_names(['ZONE'], inplace=True)


In [24]:
df_results

,BEST_PARAMS,CV,MODEL,FC,TESTSCORE,TRAINSCORE
ZONE,,,,,,
ZONE1,NaN,NaN,Baseline,NaN,0.330221,0.274462
ZONE2,NaN,NaN,Baseline,NaN,0.290033,0.256417
ZONE3,NaN,NaN,Baseline,NaN,0.312627,0.296552
ZONE4,NaN,NaN,Baseline,NaN,0.366647,0.31975
ZONE5,NaN,NaN,Baseline,NaN,0.361371,0.326282
ZONE6,NaN,NaN,Baseline,NaN,0.355799,0.331287
ZONE7,NaN,NaN,Baseline,NaN,0.307106,0.251734
ZONE8,NaN,NaN,Baseline,NaN,0.316515,0.262741
ZONE9,NaN,NaN,Baseline,NaN,0.312423,0.276838
